In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
# from plot_config import graph

In [63]:
df = pd.read_csv('/home/ramprasad.sa/factual_evaluation_source_based/datasets/scored/xformer_llm_annotated_scored_label_fewshot0.csv')
df.head()


,Unnamed: 0.1,Unnamed: 0,docid,model,nonfactual_spans,evidence,summary,factual_error,error_type,dialogue,origin,dialogue_atomic_facts,Alpaca-direct_assesment_response_fewshot,Alpaca-direct_assesment_label_fewshot,Alpaca-direct_assesment_response_fewshot_afacts,Alpaca-direct_assesment_label_fewshot_afacts,ChatGPT-direct_assesment_response_fewshot,ChatGPT-direct_assesment_label_fewshot,ChatGPT-direct_assesment_response_fewshot_afacts,ChatGPT-direct_assesment_label_fewshot_afacts
0,0,0,13728767-1,bart_large,[],[],There is a problem with the road from the swim...,0,[],Karen: you know that road from the swimming po...,FacEval,Karen talks about a road from the swimming poo...,Yes\n,0,Yes\n,0,Yes,0,Yes,0
1,1,1,13728767-1,co-ref bart large,[],[],Karen is worried about the road from the swimm...,0,[],Karen: you know that road from the swimming po...,FacEval,Karen talks about a road from the swimming poo...,Yes\n,0,Yes\n,0,Yes,0,Yes,0
2,2,2,13728767-1,condigsum bart large,[],[],There is a road blocked off from the swimming ...,0,[],Karen: you know that road from the swimming po...,FacEval,Karen talks about a road from the swimming poo...,Yes\n,0,Yes\n,0,Yes,0,Yes,0
3,3,3,13728767-1,mv-bart_large,[],[],The road from the swimming pool to Waitrose is...,0,[],Karen: you know that road from the swimming po...,FacEval,Karen talks about a road from the swimming poo...,Yes\n,0,Yes\n,0,Yes,0,Yes,0
4,4,4,13728767-1,gpt4-32k-0613,['Karen informed Peter that the road from the ...,['Peter : maybe repairs to something?'],Karen informed Peter that the road from the sw...,1,['Intrinsic_Error'],Karen: you know that road from the swimming po...,FacEval,Karen talks about a road from the swimming poo...,Yes\n,0,Yes\n,0,Yes,0,Yes,0


In [64]:
df.keys()

Index(['Unnamed: 0.1', 'Unnamed: 0', 'docid', 'model', 'nonfactual_spans',
       'evidence', 'summary', 'factual_error', 'error_type', 'dialogue',
       'origin', 'dialogue_atomic_facts',
       'Alpaca-direct_assesment_response_fewshot',
       'Alpaca-direct_assesment_label_fewshot',
       'Alpaca-direct_assesment_response_fewshot_afacts',
       'Alpaca-direct_assesment_label_fewshot_afacts',
       'ChatGPT-direct_assesment_response_fewshot',
       'ChatGPT-direct_assesment_label_fewshot',
       'ChatGPT-direct_assesment_response_fewshot_afacts',
       'ChatGPT-direct_assesment_label_fewshot_afacts'],
      dtype='object')

In [66]:
set(list(zip(df['ChatGPT-direct_assesment_fewshot_response_afacts'].values,
             df['ChatGPT-direct_assesment_fewshot_label_afacts'].values)))



KeyError: 'ChatGPT-direct_assesment_fewshot_response_afacts'

In [67]:
def calculate_auc_score(predictions, labels):
    score = roc_auc_score(labels, predictions)
    return score


In [68]:
def print_scores(df):
    factual_labels = list(df['factual_error'].values)
    factual_labels = [1 - each for each in factual_labels]

    print('DIALOGUE SCORES')
    print('-'* 13)
    for column_name in df.keys():
        if '_label' in column_name and 'afact' not in column_name:
            predictions = df[column_name].values
            flipped_predictions = [1 - each for each in predictions]
            print(column_name.split('_score')[0].strip(), 
                  max([calculate_auc_score(predictions, factual_labels),
                      calculate_auc_score(flipped_predictions, factual_labels)])
                 )
    print('***'* 13)

    print('ATOMIC SCORES')
    print('-'* 13)
    for column_name in df.keys():
        if '_label' in column_name and 'afact' in column_name:
            predictions = df[column_name].values
            flipped_predictions = [1 - each for each in predictions]
            print(column_name.split('_score')[0].strip(), 
                  max([calculate_auc_score(predictions, factual_labels),
                      calculate_auc_score(flipped_predictions, factual_labels)]))
    print('***'* 13)
        

In [69]:
llm_models = ['alpaca-13b', 'gpt4-32k-0613']
print('*' * 13, ' LLM RESULTS ', '*' * 13)
df_llm = df[df['model'].isin(llm_models)]
print_scores(df_llm)

*************  LLM RESULTS  *************
DIALOGUE SCORES
-------------
Alpaca-direct_assesment_label_fewshot 0.5012959627465624
ChatGPT-direct_assesment_label_fewshot 0.5914312700434916
***************************************
ATOMIC SCORES
-------------
Alpaca-direct_assesment_label_fewshot_afacts 0.5027566665202302
ChatGPT-direct_assesment_label_fewshot_afacts 0.6194482273865483
***************************************


In [70]:
llm_models = ['alpaca-13b', 'gpt4-32k-0613']
print('*' * 13, ' TRANSFORMER RESULTS ', '*' * 13)
df_xformer = df[~df['model'].isin(llm_models)]
print_scores(df_xformer)

*************  TRANSFORMER RESULTS  *************
DIALOGUE SCORES
-------------
Alpaca-direct_assesment_label_fewshot 0.5014075861485057
ChatGPT-direct_assesment_label_fewshot 0.7068580137160274
***************************************
ATOMIC SCORES
-------------
Alpaca-direct_assesment_label_fewshot_afacts 0.5101134535602405
ChatGPT-direct_assesment_label_fewshot_afacts 0.7073194479722293
***************************************
